In [ ]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
import csv
from resizeimage import resizeimage
from PIL import Image
from tensorflow.python.ops.image_ops_impl import ResizeMethod  
#from pandas_ml import ConfusionMatrix
from random import shuffle
import datetime

In [ ]:
def weight_variable(shape,na):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial,name=na)

def bias_variable(shape,na):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=na)

def conv2d(xe, W):
    return tf.nn.conv2d(xe, W, strides=[ 1, 1, 1, 1], padding='SAME')

def conv1d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='VALID')


def max_pool(xe):
    return tf.nn.max_pool(xe, ksize=[1, 3, 3, 1],strides=[1,2,2, 1], padding='SAME')

def max_pool_s2(xe):
    return tf.nn.max_pool(xe, ksize=[1, 3,3, 1],strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
def bn(is_training,inputs,decay=0.999):
    scale=tf.Variable(tf.ones(inputs.get_shape()[-1:]))
    beta = tf.Variable(tf.zeros(inputs.get_shape()[-1:]))
    pop_mean = tf.Variable(tf.zeros(inputs.get_shape()[-1:]), trainable=False)
    pop_var = tf.Variable(tf.ones(inputs.get_shape()[-1:]), trainable=False)
    mean = tf.cond(is_training,lambda:tf.nn.moments(inputs,[0,1,2])[0],lambda:tf.ones(tf.shape(inputs.get_shape()[-1:]))*pop_mean)
    var = tf.cond(is_training,lambda:tf.nn.moments(inputs,[0,1,2])[1],lambda:tf.ones(tf.shape(inputs.get_shape()[-1:]))*pop_var)
    train_mean = tf.cond(is_training,lambda:tf.assign(pop_mean,pop_mean * decay + mean * (1 - decay)),lambda:tf.ones([0]))
    train_var= tf.cond(is_training,lambda:tf.assign(pop_var,pop_var * decay + var * (1 - decay)),lambda:tf.ones([0]))
    with tf.control_dependencies([train_mean,train_var]):
        return tf.nn.batch_normalization(inputs,mean,var,beta,scale,epsilon)

In [ ]:
tf.reset_default_graph()
xin = tf.placeholder(tf.float32,shape=[None, 30,30,3])
y_ = tf.placeholder(tf.float32,shape=[None, 2])
epsilon=0.000001
is_training=tf.placeholder(tf.bool,[])

x_image = tf.reshape(xin, [-1, 30,30, 3])

W1=weight_variable([2,2,3,32],'w1')
b1=bias_variable([32],'b2')

h_conv1=tf.nn.relu(bn(is_training,conv2d(x_image,W1))+b1)


w2=weight_variable([2,2,32,32],'w2')
b2=bias_variable([32],'b2')
conv1=tf.nn.relu(bn(is_training,conv2d(h_conv1,w2))+b2)
conv1=max_pool(conv1)

w3=weight_variable([2,2,32,64],'w3')
b3=bias_variable([64],'b3')
conv2=tf.nn.relu(bn(is_training,conv2d(conv1,w3))+b3)

w4=weight_variable([2,2,64,64],'w4')
b4=bias_variable([64],'b4')
conv3=tf.nn.relu(bn(is_training,conv2d(conv2,w4))+b4)
conv3=max_pool(conv3)


w5=weight_variable([2,2,64,128],'w5')
b5=bias_variable([128],'b5')
conv4=tf.nn.relu(bn(is_training,conv2d(conv3,w5))+b5)

w6=weight_variable([2,2,128,128],'w6')
b6=bias_variable([128],'b6')
conv5=tf.nn.relu(bn(is_training,conv2d(conv4,w6))+b6)
conv5=max_pool(conv5)

w7=weight_variable([2,2,128,256],'w7')
b7=bias_variable([256],'b7')
conv6=tf.nn.relu(bn(is_training,conv2d(conv5,w7))+b7)

w8=weight_variable([2,2,256,256],'w8')
b8=bias_variable([256],'b8')
conv7=tf.nn.relu(bn(is_training,conv2d(conv6,w8))+b8)
#conv6=max_pool(conv6)

W_fc1=weight_variable([16*256,1000],'w_fc1')
b_fc1=bias_variable([1000],'b_fc1')
y1=tf.reshape(conv6,[-1,16*256])
y_fc1=tf.nn.relu(tf.matmul(y1,W_fc1)+b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(y_fc1, keep_prob)

W_fc2=weight_variable([1000,100],'w_fc2')
b_fc2=bias_variable([100],'b_fc2')
y_fc2=tf.nn.relu(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

W_fc3=weight_variable([100,2],'w_fc3')
b_fc3=bias_variable([3],'b_fc3')
y_fc3=tf.nn.relu(tf.matmul(y_fc2,W_fc3)+b_fc3)


In [ ]:
pred = tf.argmax(y_fc3, 1)
lab = tf.argmax(y_, 1)

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_fc3))
loss = tf.reduce_mean(cross_entropy)
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.cast(tf.equal(tf.argmax(y_fc3, 1), tf.argmax(y_, 1)),tf.float32)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as sess:
    print('training')
    a=datetime.datetime.now().time()
    print(a)
    print('session started')
    saver.restore(sess,r'../class_he_bg/model_1000.ckpt')
    #in each 'i' the model is trained with 1000 images
    
    #test_he_se=[14,18,19,25,32,38,46,47,49,54]
    test_he=[2,4,5,6,16,27,34,37,43,44]
    #aa=test_he_se+test_he
    for l11,i11 in enumerate(test_he):
        a=datetime.datetime.now().time()
        print('start time %s'%a)
        gen=np.array([])
        image=Image.open(r'../Apparent Retinopathy/IDRiD_%02d.jpg'%i11)
        g = np.array(resizeimage.resize_cover(image, [512,512]))
        gt_ex= Image.open(r'../EX/IDRiD_%02d_EX.tif'%i11)
        #gt_se= Image.open(r'/home/smst/Desktop/benjamin/SE/IDRiD_%02d_SE.tif'%i11)
        ex=np.array(resizeimage.resize_cover(gt_ex,[512,512]))
        ex=ex[15:497,15:497]
        #se=np.array(resizeimage.resize_cover(gt_se,[512,512]))
        #se=se[15:497,15:497]
        npg=np.array(g,dtype='f')
        gen=np.array([])
        for i2 in range(15,497):
            for j2 in range(15,497):
                new=npg[i2-15:i2+15,j2-15:j2+15]
                new=np.array([new])
                if tii.shape[0]%4==0:
                    tii=new
                else:
                    tii=np.append(tii,new,axis=0)
                    if tii.shape[0]%4==0:
                        pre=pred.eval(feed_dict={xin:tii,is_training:False,keep_prob: 0.5})
                        gen=np.append(gen,pre)
        n_i=np.reshape(gen,[482,482])
        axx=0
        n_gt=np.zeros([482,482])
        for h1 in range(482):
            for w1 in range(482):
                if ex[h1,w1]==1:
                    n_gt[h1,w1]=1
        for he in range(482):
            for br in range(482):
                if n_gt[he,br]==n_i[he,br]:
                    axx+=1
        acc=axx/232324.0
        n_gt=n_gt*100
        print('accuracy of %02d image is: %f'%(test_he[l11],acc))
        result1 = Image.fromarray((n_gt).astype(np.uint8))
        result1.save(r'../he_se_reco_im/he_reco_gt_%02d.jpg'%test_he[l11])
        print('%02d image gt'%test_he[l11])